In [8]:
import json
from sklearn.model_selection import train_test_split
import re

In [9]:
def convert_escape_sequences(input_string):
    # Using regular expression to replace multiple spaces with a single space
    # input_string= re.sub(r'\s+', ' ', input_string)
    return re.sub(r'[\n\r\t\f\v]', ' ', input_string)

def bio_tagging_2(words,aspects):
    labels=['O' for i in range(len(words))]
    for i in aspects:
        s=i['term']
        start=i['from']
        end=i['to'] 
        labels[start]="B"
        for j in range(start+1,end):
            labels[j]="I"
    return labels

def bio_tagging_1(values,text):
    text=convert_escape_sequences(text)
    words=text.split(" ")
    labels=['O' for i in range(len(words))]
    curr=0
    currcount=0

    for i in values:
        i=i['value']
        start=i['start']
        end=i['end']
        s=text[start:end+1].split()
        label=i['labels'][0]
        while(curr<len(words)):
            # if ' '.join(s) in ' '.join(words[curr:curr+len(s)]):
            if currcount+len(words[curr])>=start:
                labels[curr]="B_"+label
                currcount+=len(words[curr])+1
                curr+=1
                
                while(currcount<end):
                # for j in range(len(s)-1):
                    labels[curr]='I_'+label
                    currcount+=len(words[curr])+1
                    curr+=1
                break
            currcount+=len(words[curr])+1
            curr+=1
    i=0
    while(i<len(words)):
        if words[i]=="":
            words.pop(i)
            labels.pop(i)
        i+=1
    # print(words)
    return (" ".join(words),labels)
    

## NER_JUDGEMENT

In [10]:
def file_1(input,file):
    data={}
    for i in input:
        values=i['annotations'][0]['result']
        text=i['data']['text']
        words,labels=bio_tagging_1(values,text)
        data[i['id']]={"text":words,"labels":labels}
    file_path = "processed/"+file
    with open(file_path, "w") as json_file:
        json.dump(data, json_file,indent=3)

### Train and Val

In [11]:
input_file="NER_TRAIN_JUDGEMENT.json"
f = open(input_file,)
input=json.load(f)

input_data=[[],[]]
for i in input:
    values=i['annotations'][0]['result']
    text=i['data']['text']
    input_data[0].append(values)
    input_data[1].append(text)
train,val=train_test_split(input,test_size=0.15,random_state=42)

In [12]:
file_1(train,"NER_train.json")
file_1(val,"NER_val.json")

### Test

In [13]:
input_file="NER_TEST_JUDGEMENT.json"
f = open(input_file,)
test=json.load(f)
print(len(test))
file_1(test,"NER_test.json")

949


## Laptop Review

In [14]:
train=json.load(open("Laptop_Review_Train.json"))
val=json.load(open("Laptop_Review_Val.json"))
test=json.load(open("Laptop_Review_Test.json"))
def func(input,output_name):
    data={}
    count=0
    for i in input:
        words=i['words']
        aspects=i['aspects']
        text=i['raw_words']
        # print(bio_tagging_2(words,aspects))
        data[count]={"text":text,"labels":bio_tagging_2(words,aspects)}
        count+=1
        # break
    file_path = "processed/"+output_name+".json"
    # # Write data to JSON file
    with open(file_path, "w") as json_file:
        json.dump(data, json_file,indent=3)

In [15]:
func(train,"ATE_train")
func(val,"ATE_val")
func(test,"ATE_test")